In [1]:
import mediapipe as mp
import cv2
import numpy as np
# from facial_landmarks import FaceLandmarks

In [6]:
# class FaceLandmarks:
#     def __init__(self):
#         mp_face_mesh = mp.solutions.face_mesh
#         self.face_mesh = mp_face_mesh.FaceMesh()

#     def get_facial_landmarks(self, frame):
#         height, width, _ = frame.shape
#         frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         result = self.face_mesh.process(frame_rgb)

#         facelandmarks = []
#         if result.multi_face_landmarks is not None:
#             for facial_landmarks in result.multi_face_landmarks:
#                 for i in range(0, 468):
#                     pt1 = facial_landmarks.landmark[i]
#                     x = int(pt1.x * width)
#                     y = int(pt1.y * height)
#                     facelandmarks.append([x, y])
#             return np.array(facelandmarks, np.int32)
#         else: return None

class FaceLandmarks:
    def __init__(self):
        mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = mp_face_mesh.FaceMesh()

    def get_facial_landmarks(self, frame):
        height, width, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = self.face_mesh.process(frame_rgb)

        facelandmarks = []
        face_found = bool(result.multi_face_landmarks)
        if face_found:
            for facial_landmarks in result.multi_face_landmarks:
                for i in range(0, 468):
                    pt1 = facial_landmarks.landmark[i]
                    x = int(pt1.x * width)
                    y = int(pt1.y * height)
                    facelandmarks.append([x, y])
            return np.array(facelandmarks, np.int32)
        else: return None

In [7]:
fl = FaceLandmarks()
cap = cv2.VideoCapture("person_walking.mp4")
# cap = cv2.VideoCapture(0)

In [8]:
# using opencv to read the cv2

# while cap.isOpened():
#     ret, frame = cap.read()
#     frame = cv2.resize(frame, None, fx = 0.2, fy = 0.2)
#     cv2.imshow("frame", frame)
#     if cv2.waitKey(10) & 0xFF == ord("q"):
#         break
            
# cap.release()
# cv2.destroyAllWindows()

In [9]:
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5)
    frame_copy = frame.copy()
    height, width, _ = frame.shape


    # 1. Face landmarks detection
    
    landmarks = fl.get_facial_landmarks(frame)
    
    if landmarks is not None:
        convexhull = cv2.convexHull(landmarks)

        # 2. Face blurrying
        mask = np.zeros((height, width), np.uint8)
        # cv2.polylines(mask, [convexhull], True, 255, 3)
        cv2.fillConvexPoly(mask, convexhull, 255)

        # Extract the face
        frame_copy = cv2.blur(frame_copy, (34, 34))
        face_extracted = cv2.bitwise_and(frame_copy, frame_copy, mask=mask)


        # Extract background
        background_mask = cv2.bitwise_not(mask)
        background = cv2.bitwise_and(frame, frame, mask=background_mask)

        # Final result
        result = cv2.add(background, face_extracted)

        #cv2.imshow("burrred", face_extracted)
        cv2.imshow("Result", result)
        # cv2.imshow("Frame", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        cv2.imshow("Result", frame)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break


cap.release()
cv2.destroyAllWindows()

In [9]:
cap.release()
cv2.destroyAllWindows()